In [23]:
# Python standard libraries
import math
import random
import os

# Packages for language model
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict

# Packages for computation and modelling
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm
import mne
import pickle

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Self-defined packages
from swlda import SWLDA
from utils import *

# Magic command to reload packages whenever we run any later cells
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package cmudict to
[nltk_data]     /Users/zionshane/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [24]:
BOARD = [["A",    "B",  "C",   "D",    "E",    "F",     "G",    "H"    ],
         ["I",    "J",  "K",   "L",    "M",    "N",     "O",    "P"    ],
         ["Q",    "R",  "S",   "T",    "U",    "V",     "W",    "X"    ],
         ["Y",    "Z",  "Sp",  "1",    "2",    "3",     "4",    "5"    ],
         ["6",    "7",  "8",   "9",    "0",    "Prd",   "Ret",  "Bs"   ],
         ["?",    ",",  ";",   "\\",   "/",    "+",     "-",    "Alt"  ],
         ["Ctrl", "=",  "Del", "Home", "UpAw", "End",   "PgUp", "Shft" ],
         ["Save", "'",  "F2",  "LfAw", "DnAw", "RtAw",  "PgDn", "Pause"],
         ["Caps", "F5", "Tab", "EC",   "Esc",  "email", "!",    "Sleep"]]
BOARD  = np.array(BOARD)
N_ROWS = BOARD.shape[0]  # number of rows
N_COLS = BOARD.shape[1]  # number of columns
M = N_ROWS * N_COLS      # the number of chars on the board

In [25]:
paradigm       = 'RC'  # display paradigm ('RC', 'CB', or 'RD')
NUM_TIMESTAMPS = 195   # number of timestamps in each window to record signals
EPOCH_SIZE     = 15    # required number of features in every epoch
CORE_CHANNELS = ('EEG_Fz', 'EEG_Cz',  'EEG_P3',  'EEG_Pz',
                 'EEG_P4', 'EEG_PO7', 'EEG_PO8', 'EEG_Oz')
NUM_CORE_CHANNELS  = len(CORE_CHANNELS)  # number of core eletrodes
NUM_SEQ = 10        # max number of sequences (all rows and cols are flashed)
T_MAX = (N_ROWS + N_COLS) * NUM_SEQ # max number of flashes

In [26]:
f = open(f'./model/D02-model.pkl', 'rb')
clf = pickle.load(f)

In [44]:
obj = '02'
run = '01'

In [45]:
directory = '/Users/zionshane/Desktop/Duke/Research/BCI_data/EDFData-StudyD'
obj_directory = directory + f'/D{obj}/SE001'
test_file = obj_directory+f'/Test/Dyn/D{obj}_SE001Dyn_Test{run}.edf'
raw_data = mne.io.read_raw_edf(test_file, preload=True, verbose=False)

stim_events = mne.find_events(raw=raw_data,
                              stim_channel='StimulusBegin',
                              verbose=False)
eeg_channels = mne.pick_channels_regexp(raw_data.info['ch_names'], 'EEG')
raw_data.notch_filter(freqs=60, picks=eeg_channels, verbose=False)
test_epochs = get_core_epochs(raw_data)

In [46]:
current_target_events = mne.find_events(raw_data, stim_channel='CurrentTarget',
                                        verbose=False)
current_target_appears = current_target_events[:,0]
current_target = current_target_events[:,2]
truth = eventIDs_to_sequence(BOARD, current_target)
str_truth = ''.join(truth)
print(f'Participant #{obj} wants to type "{str_truth}" in test #{run}.')

Participant #02 wants to type "PIECES" in test #01.


In [47]:
selected_target_events = mne.find_events(raw_data, stim_channel='SelectedTarget',
                                        verbose=False)
selected_target_appears = selected_target_events[:,0]
selected_target = selected_target_events[:,2]
selected = eventIDs_to_sequence(BOARD, selected_target)
str_selected = ''.join(selected)
print(f'Participant #{obj} typed "{str_selected}" in test #{run}.')

Participant #02 typed "PauseQ0CEC" in test #01.
